# CSCI236 assignment 1 Task1 
* Name: Nguyen Gia Khanh
* UOWID: 7311217


### Import libraries

In [ ]:
import pandas as pd
import numpy as np

### 1. Read csv file as pandas dataframe

In [ ]:
df1 = pd.read_csv("customer_churn_dataset-training-master.csv")
df2 = pd.read_csv("customer_churn_dataset-testing-master.csv")
df = pd.concat([df1, df2], axis=0)
df.info()

### 2. Identify missing values

In [ ]:
# get general info about test dataset
df.isnull().sum()

Check for missing value in train dataset

In [ ]:
missing_value = df[df.isnull().any(axis=1)]
missing_value

In train dataset, row 199295 contains missing values <br>
We can safely remove the whole data point because: <br>
- This is the only record that has missing value (<5% dataset's size)
- It missing all of its attribute


In [ ]:
df = df.dropna()

### 3. Using z-score normalization for column "Last Interaction"

In [ ]:
train_mean_li = df["Last Interaction"].mean()
train_std_li = df["Last Interaction"].std()
df["LI z-score"] = (df["Last Interaction"] - train_mean_li)/train_std_li
# Swap the values and column names
df['Churn'], df['LI z-score']  = df['LI z-score'], df['Churn']
df.rename(columns={'Churn': 'LI z-score', 'LI z-score': 'Churn'}, 
                    inplace=True)
df.head()

Mean of normalized data

In [ ]:
round(df["LI z-score"].mean())

Variance of normalized data

In [ ]:
df["LI z-score"].var()

### 4. Create five bins for the attribute “Total Spend” such that the bins contain (approximately) equivalent numbers of records

In [ ]:
bin_labels = ['Bin1', 'Bin2', 'Bin3', 'Bin4', 'Bin5']
df['Total Spend'] = pd.cut(df['Total Spend'], bins=5, labels=bin_labels)

### 5.  Apply one-hot-encoding to the attribute “Contract Length”


In [ ]:
ohe = pd.get_dummies(df["Contract Length"]).astype(int)
df = pd.concat([df, ohe], axis=1)
df = df.drop(columns=["Contract Length"])
# Specify the desired column order
desired_order = [
    'CustomerID', 'Age', 'Gender', 'Tenure', 'Usage Frequency', 'Support Calls',
    'Payment Delay', 'Subscription Type', 'Total Spend',
    'Last Interaction', 'LI z-score', 'Annual', 'Monthly', 'Quarterly','Churn'
]

# Reorder the columns in the DataFrame
df = df[desired_order]
df

### 6. Define at least one new attribute based on existing attribute, and explain your reason behind your definition

Attribute purpose: Support request frequency <br>
---
Measured by: Support Calls / Tenure <br>
---
Explanation:<br>
<ul>
<li> By dividing the total support calls by the customer tenure, we can know how often a customer reach for assistance while using the service.
<li> The more often customer requests support, the more frustration they may experience, and this can be a good attribute to measure customer churn.
</ul>

In [ ]:
df["Support request frequency"] = df["Support Calls"] / df["Tenure"]
df['Churn'], df['Support request frequency']  = df['Support request frequency'], df['Churn']
df.rename(columns={'Churn': 'Support request frequency', 'Support request frequency': 'Churn'}, 
                    inplace=True)

In [ ]:
df.tail()